In [23]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

sentences = [ "i like dog", "i love coffee", "i hate milk","i love you","i love coffee","i love you"]

class LanguageModel:
    def __init__(self,sentences):
        self.sentences = sentences
    
    def getWordsList(self):
        words_list = self.sentences
        word_list = " ".join(sentences).split()
        word_list = list(set(word_list))
        return word_list
        
    def getWordsDict(self):
        words_list = self.getWordsList()
        word_dict = {w: i for i, w in enumerate(word_list)}
        number_dict = {i: w for i, w in enumerate(word_list)}
        return word_dict,number_dict
                
    def make_batch(self):
        sentences = self.sentences
        input_batch = []
        target_batch = []

        for sen in sentences:
            word = sen.split()
            input = [word_dict[n] for n in word[:-1]]
            target = word_dict[word[-1]]

            input_batch.append(np.eye(n_class)[input])
            target_batch.append(np.eye(n_class)[target])

        return input_batch, target_batch
    
LM = LanguageModel(sentences)
words_list = LM.getWordsList()
word_dict,number_dict = LM.getWordsDict()
n_class = len(word_dict)


# NNLM Parameter
n_step = 2 # number of steps ['i like', 'i love', 'i hate']
n_hidden = 2 


In [24]:
# Model
X = tf.placeholder(tf.float32, [None, n_step, n_class]) # [batch_size, number of steps, number of Vocabulary]
Y = tf.placeholder(tf.float32, [None, n_class])

input = tf.reshape(X, shape=[-1, n_step * n_class]) # [batch_size, n_step * n_class]
H = tf.Variable(tf.random_normal([n_step * n_class, n_hidden]))
d = tf.Variable(tf.random_normal([n_hidden]))
U = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

tanh = tf.nn.tanh(d + tf.matmul(input, H)) # [batch_size, n_hidden]
model = tf.matmul(tanh, U) + b # [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction =tf.argmax(model, 1)

In [25]:
# Trainingyo
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = LM.make_batch()

for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))


Epoch: 1000 cost = 1.118109
Epoch: 2000 cost = 0.598899
Epoch: 3000 cost = 0.503481
Epoch: 4000 cost = 0.481435
Epoch: 5000 cost = 0.472652


In [26]:
# Predict
predict =  sess.run([prediction], feed_dict={X: input_batch})

In [27]:
# Test
input = [sen.split()[:2] for sen in sentences]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n] for n in predict[0]])

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['i', 'love'], ['i', 'love'], ['i', 'love']] -> ['dog', 'coffee', 'milk', 'coffee', 'coffee', 'coffee']
